# Step 1 is to setup the Google collab environment 

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2 
!nvcc --version
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
from google.colab import drive
drive.mount("/content/gdrive")


In [ ]:
%%writefile main.cu
#include "/usr/include/opencv2/opencv.hpp"
#include <iostream>
#include <vector>
using namespace cv;
using namespace std;

#define DIM 1000

struct cuComplex { 
    
  float r;
  float i;
  __device__ cuComplex(float a, float b): r(a), i(b) {}

  __device__ float magnitude(void) {
      return r * r + i * i;
  }
  __device__ cuComplex operator*(cuComplex& a) {
      return cuComplex(r * a.r - i*a.i, i * a.r + r*a.i);
  }

  __device__ cuComplex operator+(const cuComplex& a) {
      return cuComplex(r + a.r, i + a.i);
  }
};


__device__  bool is_in_julia_set(int x, int y) {

  const float scale = 1.5;
  float cx = scale * (float) (DIM/2 - x) / (DIM/2); 
  float cy =  scale * (float) (DIM/2 - y) / (DIM/2);

  cuComplex constant(-0.8, 0.156);
  cuComplex c(cx, cy);

  for (int i = 0; i < 200; i++) {

      c = (c * c) + constant;
      if (c.magnitude() > 10000) {
          return false;
      }
  }

  return true;
}

__global__ void kernel(uchar* d_grid) { 

  int x, y;
  x = blockIdx.x;
  y = blockIdx.y;
  int index = x + (y * DIM);

  if (is_in_julia_set(x, y)){ 
      d_grid[index] = (uchar)0;
  }
  else {
      d_grid[index] = (uchar)255;
  }
}


int main(int argc, char** argv) {
  
  int grid_size = sizeof(uchar) * DIM * DIM;
  uchar* h_grid = (uchar*)malloc(grid_size);
  uchar* d_grid;

  cudaMalloc((void**)&d_grid, grid_size);

  dim3 grid(DIM, DIM, 1);
  kernel<<<grid, 1>>>(d_grid);

  cudaDeviceSynchronize();
  cudaMemcpy(h_grid, d_grid, grid_size, cudaMemcpyDeviceToHost);

  cudaFree(d_grid);
  
  Mat image = Mat(DIM, DIM, CV_8UC1, h_grid);
  imwrite("julia_set.jpg", image);
}

In [ ]:
%%script bash
nvcc main.cu -o main.cu `pkg-config --cflags --libs opencv`

In [ ]:
%%script bash
./main.cu

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
img = cv.imread('julia_set.jpg')
plt.imshow(img)